In [1]:
import torch 
import torch.nn.functional as F 
import matplotlib.pyplot as plt

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()

In [3]:
len(words)

32033

In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [5]:
vocab_size = len(itos)
print(vocab_size)

27


In [6]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


### Batch-Normalization 
* The BN helps to train very deep NN quite reliable

* This takes to pre-activation values and normalize them to have unit variance [@ each neuron] , without worrying about the initial weights 

* When we are using batch normalization we don't need to bias weights 

* BN couples inputs in the batch via batch mean and batch std -> this provides some regularization as secondary effect 

In [7]:
# Let's make the code more similar to pytorch implimentation 

class Linear:
    def __init__(self,fan_in,fan_out,bias=True):
        self.weight = torch.randn((fan_in,fan_out),generator=g)/fan_in**0.5
        self.bias = torch.zeros(fan_out)
        
    def __call__(self,x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])
    

class BatchNorm1d:
    def __init__(self,dim,eps=1e-5,momentum=0.1): # here dim is the hidden layer dim 
        self.eps = eps
        self.momentum = momentum 
        self.training = True 
        self.gamma = torch.ones(dim)
        self.beta  = torch.zeros(dim)
        self.running_mean = torch.zeros(dim)
        self.running_var  = torch.ones(dim)
        
    def __call__(self,x):
        if self.training:
            xmean = x.mean(0,keepdim=True)
            xvar  = x.var(0,keepdim=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean)/torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var  = (1 - self.momentum) * self.running_var  + self.momentum * xvar
                
        return self.out 
    
    def parameters(self):
        return [self.gamma, self.beta]
    
class Tanh:
    def __call__(self,x):
        self.out = torch.tanh(x)
        return self.out 
    def parameters(self):
        return []

In [8]:
n_embd = 10 
n_hidden = 100 
block_size = 3 
vocab_size = 27
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size,n_embd),generator=g)

layers = [
    Linear(n_embd*block_size,n_hidden,bias=False),BatchNorm1d(n_hidden),Tanh(),
    Linear(         n_hidden,n_hidden,bias=False),BatchNorm1d(n_hidden),Tanh(),
    Linear(         n_hidden,n_hidden,bias=False),BatchNorm1d(n_hidden),Tanh(),
    Linear(         n_hidden,n_hidden,bias=False),BatchNorm1d(n_hidden),Tanh(),
    Linear(         n_hidden,n_hidden,bias=False),BatchNorm1d(n_hidden),Tanh(),
    Linear(         n_hidden,vocab_size,bias=False),BatchNorm1d(vocab_size),
]

with torch.no_grad():
    layers[-1].gamma *=0.1      # make last layer less confident 
    for layer in layers[:-1]:   # all the ohther layers: apply gain 
        if isinstance(layer,Linear):
            layer.weight *= 0.1 # 5/3 
parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters))

for p in parameters:
    p.requires_grad = True 

47551


In [9]:
ix = torch.randint(0,Xtr.shape[0],(32,),generator=g)
Xb, Yb = Xtr[ix],Ytr[ix]

print(Xb.shape)
print(C.shape)
print(C[Xb].shape)

torch.Size([32, 3])
torch.Size([27, 10])
torch.Size([32, 3, 10])


In [10]:
# optimization 
max_steps = 200000
batch_size = 32 
lossi = []
ud = []

for i in range(max_steps):
    # mini batch 
    ix = torch.randint(0,Xtr.shape[0],(batch_size,),generator=g)
    Xb, Yb = Xtr[ix],Ytr[ix]
    
    # forward pass 
    emb = C[Xb] # embed chars to vectors 
    x = emb.view(emb.shape[0],-1) # concatenate the vectors 
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x,Yb)
    
    # backward pass 
    for layer in layers:
        layer.out.retain_grad()
    for p in parameters:
        p.grad = None 
    loss.backward()
    
    # update 
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr*p.grad
    
    # track status 
    if i % 10000 == 0:
        print(f'{i:7d}/{max_steps:7d}:{loss.item():.4f}')
    lossi.append(loss.log10().item())
    
    with torch.no_grad():
        ud.append([((lr*p.grad).std()/p.data.std()).log10().item() for p in parameters])
        
    

      0/ 200000:3.2912
  10000/ 200000:2.2274
  20000/ 200000:1.9601
  30000/ 200000:2.1760
  40000/ 200000:2.0386
  50000/ 200000:2.4513
  60000/ 200000:2.0496
  70000/ 200000:2.1536
  80000/ 200000:2.0176
  90000/ 200000:2.1248
 100000/ 200000:2.3377
 110000/ 200000:2.1634
 120000/ 200000:2.1111
 130000/ 200000:1.9837
 140000/ 200000:2.0169
 150000/ 200000:2.1802
 160000/ 200000:2.5559
 170000/ 200000:2.2282
 180000/ 200000:2.2986
 190000/ 200000:2.1221


In [11]:
# inference 
@torch.no_grad()
def split_loss(split):
    x,y = {
        'train':(Xtr,Ytr),
        'val':(Xdev,Ydev),
        'test':(Xte,Yte)
    }[split]
    
    emb = C[x]
    x = emb.view(emb.shape[0],-1)
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x,y)
    print(split,loss.item())
    
# put layers into eval mode
for layer in layers:
    layer.training = False
    
split_loss('train')
split_loss('val')

train 2.0281624794006348
val 2.091705322265625


In [12]:
# sample from model 

for _ in range(10):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        x = emb.view(emb.shape[0],-1)
        for layer in layers:
            x = layer(x)
        logits = x 
        probs = F.softmax(logits)
        ix = torch.multinomial(probs,num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix ==0:
            break
    print(''.join(itos[i] for i in out))

abillie.
fnaida.
kie.
vivia.
nous.
kinlsiah.
bro.
eishis.
mikynn.
adden.


/tmp/ipykernel_4657/891351972.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(logits)
